![](https://images.pexels.com/photos/917732/pexels-photo-917732.jpeg?auto=compress&cs=tinysrgb&dpr=2&h=750&w=1260)


# Serialization: MsgPack vs Pickle

Kevin J. Walchko, Phd

23 Feb 2019

---

In [84]:
import msgpack
import pickle
import time
import timeit
from collections import namedtuple

class vec_t(namedtuple('vec_t', 'x y z')):
    __slots__ = ()

    def __new__(cls, x,y,z):
        cls.id = 0
        return cls.__bases__[0].__new__(cls,x,y,z)

class imu_st(namedtuple('imu_st', 'linear_accel angular_vel magnetic_field timestamp')):
    """
    Inertial measurement unit
    """
    __slots__ = ()

    def __new__(cls, a, g, m, ts=None):
        cls.id = 10
        if ts:
            return cls.__bases__[0].__new__(cls, a, g, m, ts)
        else:
            return cls.__bases__[0].__new__(cls, a, g, m, time.time())

# data
v = vec_t(-1,1000,0.123456)
imu = imu_st(v,v,v)

# performance function
def perf(proto, data):
    o = proto.dumps(imu)
    sz = len(o)
    print(">> packed len:", sz, "bytes")
    print(">> The same:",imu == proto.loads(o))
#     print(proto.loads(o))
    
    loops = 100000
    start = time.time()
    for i in range(loops):
        assert(data == proto.loads(proto.dumps(data)))
    totaltime = time.time() - start
    print(">> time: {:.2f} sec {} loops: {:.2f} msg/sec bandwidth: {} kb".format(
        totaltime,
        loops,
        loops/totaltime,
        sz*loops/1000)
    )

# Pickle

The pickle module implements binary protocols for serializing and de-serializing a Python object structure. Unfortunately, pickle is **python only**. [ref](https://docs.python.org/3/library/pickle.html)

In [85]:
perf(pickle, imu)

>> packed len: 83 bytes
>> The same: True
>> time: 1.16 sec 100000 loops: 85982.68 msg/sec bandwidth: 8300.0 kb


# MsgPack

MessagePack is an efficient binary serialization format. MessagePack is also **cross platform**. [ref](https://msgpack.org/index.html)

In [86]:
class MsgPack(object):
    msgs = {
        0: vec_t,
        10: imu_st
    }

    def ext_pack2(self, x):
        try:
#             return msgpack.ExtType(x.id, msgpack.packb(list(x[:]), default=self.ext_pack2, strict_types=True))
            return msgpack.ExtType(x.id, msgpack.packb(x[:], default=self.ext_pack2, strict_types=True))
        except:
            return x

    def ext_unpack2(self, code, data):
        if code in self.msgs.keys():
            d = msgpack.unpackb(data, ext_hook=self.ext_unpack2, use_list=False,raw=False)
            return self.msgs[code](*d)
        return msgpack.ExtType(code, data)

    def dumps(self, data):
        return msgpack.packb(data, use_bin_type=True, strict_types=True,default=self.ext_pack2)

    def loads(self, data):
        return msgpack.unpackb(data, use_list=False,raw=False, ext_hook=self.ext_unpack2)


In [87]:
perf(mp, imu)

>> packed len: 64 bytes
>> The same: True
>> time: 3.23 sec 100000 loops: 30962.28 msg/sec bandwidth: 6400.0 kb


# Results

**Each time the cells above are run, you will get a slightly different answer**

Using the numbers below and using `pickle` as the standard python serialization method, we see `msgpack` is **2.8 times slower** but produces serialized data **23% smaller** than `pickle`. Now this makes sense, you typically get smaller data compression the more time you spend packing it.

| Protocol|Msg Size (b)|100k msgs (sec)|Speed (msg/sec)|Bandwidth (kb)|20Hz (kbps)|20Hz (usec)|
|---------|------------|----------------|-----------------|----------------|---|---|
|`pickle` | 83 | 1.16 | 85,983 | 8300 | 1.66 | 232.60 |
|`msgpack`| 64 | 3.24 | 30,227 | 6400 | 1.28 | 661.66 |


In [102]:
speed = 3.24/1.15
size = 64/83

print("speed ratio: {:.2f}".format(speed))
print("size ratio: {:.2f}".format(size))
print('-'*40)
# these are for a different number of messages/sec ... meaningless?
print("pickle max (~86k) bw/sec: {:.2f} kbps".format(8300/1.16))
print("msgpack max (~30k) bw/sec: {:.2f} kbps".format(6400/3.24))
print('-'*40)
print("pickle 20Hz: {:.2f} kbps   {:.2f} usec".format(83*20/1000, 20/85983*1e6))
print("msgpack 20Hz: {:.2f} kbps   {:.2f} usec".format(64*20/1000, 20/30227*1e6))

speed ratio: 2.82
size ratio: 0.77
----------------------------------------
pickle max (~86k) bw/sec: 7155.17 kbps
msgpack max (~30k) bw/sec: 1975.31 kbps
----------------------------------------
pickle 20Hz: 1.66 kbps   232.60 usec
msgpack 20Hz: 1.28 kbps   661.66 usec
